In [ ]:
import os
import base64
from pymongo import MongoClient
from faker import Faker

# MongoDB connection
client = MongoClient('mongodb://localhost:27017/')
db = client['AI-RECRUITMENT']
collection = db['CANDIDATES']

# Initialize Faker
fake = Faker()

# Function to encode DOCX file to base64
def encode_docx_to_base64(docx_path):
    with open(docx_path, "rb") as docx_file:
        encoded_string = base64.b64encode(docx_file.read()).decode('utf-8')
    return encoded_string

# Path to the directory containing the DOCX files
docx_directory = '/Users/sakshigupta/Downloads/Resumes'

# Iterate over all DOCX files in the directory
for docx_file in os.listdir(docx_directory):
    if docx_file.endswith('.docx'):
        docx_path = os.path.join(docx_directory, docx_file)
        encoded_docx = encode_docx_to_base64(docx_path)
        
        # Generate fake data for additional fields
        candidate_name = fake.name()
        candidate_id = fake.numerify(text="####")
        email = candidate_name.lower().replace(" ", ".") + "@gmail.com"
        phone_number = fake.numerify(text="##########")
        address = fake.address()
        experience = fake.numerify(text="#") + ' year(s)'
        
        # Prepare the document for MongoDB
        document = {
            "id": candidate_id,
            "name": candidate_name,
            "email": email,
            "phone_number": phone_number,
            "address": address,
            "experience": experience,
            "resume": encoded_docx,
            "cover_letter": encoded_docx
        }
        
        # Insert the document into MongoDB
        collection.insert_one(document)
        print(f"Inserted {candidate_name}'s resume into MongoDB with additional fields")

print("All DOCX files have been processed and inserted into MongoDB with additional fields.")


In [11]:
import os
import base64
import pandas as pd
from pymongo import MongoClient
from faker import Faker
import random
from bs4 import BeautifulSoup

# MongoDB connection
client = MongoClient('mongodb://localhost:27017/')
db = client['AI-RECRUITMENT']
collection = db['CANDIDATES']
db.drop_collection(collection)

# Initialize Faker
fake = Faker()

# Load the dataset
file_path = '/Users/sakshigupta/Downloads/Resume.csv'  # Update this to the correct path if necessary
df = pd.read_csv(file_path)

# Function to generate a synthetic cover letter
def generate_cover_letter(candidate_name, experience, skills, relevant_field):
    technical_skills = ", ".join(skills)
    sentences = [
        f"My name is {candidate_name}, and I am passionate about {relevant_field}.",
        f"With {experience} of experience, I have honed my skills in {technical_skills}.",
        f"I excel at {fake.bs()} and {fake.bs()}, which I believe are critical for this role.",
        f"I am known for my {fake.bs()} and my ability to {fake.bs()} under pressure.",
        f"I look forward to bringing my unique blend of skills and experience to your esteemed company.",
        f"Thank you for considering my application. I am excited about the possibility of contributing to your team."
    ]
    random.shuffle(sentences)
    cover_letter = "\n".join(sentences)
    return cover_letter

# Function to insert the name into HTML resume
def insert_name_in_html(html_str, name):
    soup = BeautifulSoup(html_str, 'html.parser')
    name_tag = soup.new_tag("h1")
    name_tag.string = name
    if soup.body:
        soup.body.insert(0, name_tag)
    else:
        soup.insert(0, name_tag)
    return str(soup)

# Iterate over the dataset
for index, row in df.iterrows():
    resume_id = row['ID']
    resume_str = row['Resume_str']
    resume_html = row['Resume_html']
    category = row['Category']
    
    candidate_name = fake.name()
    email = candidate_name.lower().replace(" ", ".") + "@gmail.com"
    phone_number = fake.numerify(text="##########")
    address = fake.address()
    experience = fake.numerify(text="#") + ' year(s)'
    skills = [fake.job() for _ in range(3)]
    relevant_field = category  # Assuming category is the relevant field

    # Generate a synthetic cover letter
    cover_letter = generate_cover_letter(candidate_name, experience, skills, relevant_field)
    encoded_cover_letter = base64.b64encode(cover_letter.encode('utf-8')).decode('utf-8')

    # Insert the candidate's name into the HTML resume
    updated_resume_html = insert_name_in_html(resume_html, candidate_name)
    
    # Prepare the document for MongoDB
    document = {
        "id": resume_id,
        "name": candidate_name,
        "email": email,
        "phone_number": phone_number,
        "address": address,
        "experience": experience,
        "category": category,
        "resume_str": resume_str,
        "resume_html": updated_resume_html,
        "cover_letter": encoded_cover_letter
    }
    
    # Insert the document into MongoDB
    collection.insert_one(document)
    print(f"Inserted {candidate_name}'s resume into MongoDB with additional fields")

print("All resumes have been processed and inserted into MongoDB with additional fields.")


Inserted Gabriel Wright's resume into MongoDB with additional fields
Inserted Edward Santana's resume into MongoDB with additional fields
Inserted Erik Mason's resume into MongoDB with additional fields
Inserted Deborah Bishop's resume into MongoDB with additional fields
Inserted Gregory Rogers's resume into MongoDB with additional fields
Inserted Philip Miller's resume into MongoDB with additional fields
Inserted Melissa Garrett's resume into MongoDB with additional fields
Inserted Robert Thompson's resume into MongoDB with additional fields
Inserted Shelby Johns's resume into MongoDB with additional fields
Inserted Kent Garza's resume into MongoDB with additional fields
Inserted Robert Davis's resume into MongoDB with additional fields
Inserted Jeremy Jensen's resume into MongoDB with additional fields
Inserted Jessica Flores's resume into MongoDB with additional fields
Inserted James Cooper's resume into MongoDB with additional fields
Inserted Michele Stevenson's resume into MongoDB

In [10]:
import os
import base64
import pandas as pd
from pymongo import MongoClient
from faker import Faker
import random
from bs4 import BeautifulSoup

fake = Faker()

client = MongoClient('mongodb://localhost:27017/')
db = client['AI-RECRUITMENT']
company_name = 'Compnay 1'
collection = db[company_name]


def generate_users(count):
    users = []
    for i in range(count):
        name = fake.name()
        data = {

            'id': fake.numerify(text='####'),
            'name': name,
            'email': company_name.lower().replace(" ",".") + '@' + company_name.lower().replace(" ","-") + ".com",
            'phone': fake.phone_number(),
            'role': 'Hiring Manager',
            'company': company_name
        }
        users.append(data)

    return users

def main():
    users = generate_users(50)

    collection.insert_many(users)
    
    print('Data inserted successfully')

if __name__ == '__main__':
    main()

Data inserted successfully


In [9]:
"""
Code to insert the jobs into the database
job dataset used : https://www.kaggle.com/datasets/ravindrasinghrana/job-description-dataset
"""


import pandas as pd
from pymongo import MongoClient

file_path = '/Users/sakshigupta/Desktop/FYP/AI-RecruitmentTool/back-end/job_descriptions.csv'  
df = pd.read_csv(file_path)

client = MongoClient('mongodb://localhost:27017/')
db = client['AI-RECRUITMENT']
collection = db['JOBS']

# print(df[0])

# Insert each row in the DataFrame as a document in MongoDB
for index, row in df.iterrows():
    # Convert the row to a dictionary
    data = row.to_dict()
    # Insert the dictionary into MongoDB
    collection.insert_one(data)
    print(f'Inserted row {index}')

print('All rows inserted')

Inserted row 0
Inserted row 1
Inserted row 2
Inserted row 3
Inserted row 4
Inserted row 5
Inserted row 6
Inserted row 7
Inserted row 8
Inserted row 9
Inserted row 10
Inserted row 11
Inserted row 12
Inserted row 13
Inserted row 14
Inserted row 15
Inserted row 16
Inserted row 17
Inserted row 18
Inserted row 19
Inserted row 20
Inserted row 21
Inserted row 22
Inserted row 23
Inserted row 24
Inserted row 25
Inserted row 26
Inserted row 27
Inserted row 28
Inserted row 29
Inserted row 30
Inserted row 31
Inserted row 32
Inserted row 33
Inserted row 34
Inserted row 35
Inserted row 36
Inserted row 37
Inserted row 38
Inserted row 39
Inserted row 40
Inserted row 41
Inserted row 42
Inserted row 43
Inserted row 44
Inserted row 45
Inserted row 46
Inserted row 47
Inserted row 48
Inserted row 49
Inserted row 50
Inserted row 51
Inserted row 52
Inserted row 53
Inserted row 54
Inserted row 55
Inserted row 56
Inserted row 57
Inserted row 58
Inserted row 59
Inserted row 60
Inserted row 61
Inserted row 62
In

KeyboardInterrupt: 